In [ ]:
import torch

reward_model_path = "CartPole-v1_reward_model_seed42.pth"

reward_model = torch.load(reward_model_path, map_location="cpu")
reward_model.eval()
reward_model.to("cpu")



C:\Users\MATH-286-Dell\AppData\Local\Temp\ipykernel_41760\2129410092.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  reward_model = torch.load(reward_model_path)


AttributeError: 'collections.OrderedDict' object has no attribute 'eval'